In [ ]:
def get_matlab_data(file_path, is_label: bool):
    """reading a .mat file to extract the images or the labels"""
    print(f"Started reading {file_path} . . . ")
    data1 = h5py.File(file_path, 'r')
    data_key =  list(data1.keys())[1]

    refs=data1[data_key][0][:] # get object references
    num_of_examples = len(refs)
    if is_label: # Check if the data to read is a label
        np_array_data =  np.array([''.join(map(chr, np.squeeze(data1[refs[i]]))) for i in range(num_of_examples)]) # Taking all
    else:
        np_array_data =  np.array([np.transpose(np.array(data1[refs[i]])) for i in range(num_of_examples)]) # Taking all 

    data1.close()
    print(f"Completed reading {file_path}")

    return np_array_data


In [ ]:
# Get features
train_data = get_matlab_data('Data/train_data.mat', is_label=False)
validation_data = get_matlab_data('Data/val_data.mat', is_label=False)
test_data = get_matlab_data('Data/test_data.mat', is_label=False)